# Свертка данных. Ядерное сглаживание.

In [ ]:
%matplotlib inline

import numpy as np
import scipy.linalg as la
import scipy.interpolate as interp
import scipy.integrate as integrate
import scipy.fftpack as fft
import scipy.signal as signal
import matplotlib as mpl
import matplotlib.pyplot as plt

default_dpi = mpl.rcParamsDefault['figure.dpi']
factor = 1.5
mpl.rcParams['figure.dpi'] = default_dpi * factor

## Исходные данные

In [ ]:
x = np.linspace(0, np.pi * 4, 101)
y = 0. + 2.*np.sin(x * 2) + np.cos(x * 5 ) #+ np.sin(x * 25.5)

n = len(x)

print (x/(np.pi * 2))

yd = np.zeros(len(x))

plt.plot(x, y)
Delta = 1.0 * np.random.randn(len(x))
yd = y + Delta
y_av = np.sum(y)/len(x)

plt.figure ()

plt.plot (x, yd, 'o')

In [ ]:
Conv = np.zeros ((len(x)))
W = np.zeros ((2*len(x)))

W [-2] = 1./5
W [-1] = 1./5.
W [0] = 1./5.
W [1] = 1./5.
W [2] = 1./5.

for i in range (0, n-1):
    
    Conv [i] = 0.
    
    for k in range (0, n-1):
        Conv[i] = Conv[i] + yd[k]*W[i-k]
        
for i in range (2, 3):
        print (i, Conv[i], np.sum(yd[(i-2):(i+3)])/5.)

plt.figure ()
plt.plot (x, y)
plt.plot (x, yd, 'v')
plt.plot (x, Conv, '--')

# Автокорреляция

## Исходные данные

In [ ]:
x = np.linspace(0, np.pi * 6, 101)

n = len(x)

L = max(x) - min(x)

y = 0. + 1.*np.sin(x * 2) + 2.*np.sin(x * 4 ) #+ np.sin(x * 25.5)

#y = 1.0 * np.random.randn(len(x))

#x = np.linspace(-10, 10, 101)
#y = 5*np.exp(-10*x**2) * 0.

#y[x>-1.]=1
#y[x>1.]=0

#y[x>np.pi * 2] = 0.

#y[x>np.pi * 4] = 0.5 *np.sin(x [x>np.pi * 4]* 5)

print (x/(np.pi))

plt.plot(x/np.pi, y)

## Автокорреляционная функция

In [ ]:
Ac = np.zeros (n)

y_av = np.sum(y)/n

print (y_av)

for k in range (0, n-1):
    
    Ac [k] = 0.
    
    for i in range (0, n-k-1):
        Ac[k] = Ac[k] + (y[i]-y_av)*(y[i+k]-y_av)/(n-k-1)
        
plt.plot (x[0:int(n/2)]/L, Ac[0:(int(n/2))]/Ac[0])       


# Оконное преобразование Фурье

## Исходные данные

In [ ]:
x = np.linspace(0, np.pi * 6, 101)

n = len(x)

L = max(x) - min(x)

y = 0. + 0.5*np.sin(x * 2) #+ 2.*np.sin(x * 4 ) #+ np.sin(x * 25.5)

y[x>np.pi * 2] = 0.

y[x>np.pi * 4] = 1. *np.sin(x [x>np.pi * 4]* 4)

print (x/(np.pi))

plt.plot(x/np.pi, y)

## Подвижное окно

In [ ]:
B = np.zeros ((len(x)))

dx = L / (n-1)

xw = np.linspace(-L, L, 201)

Win = np.zeros ((2*len(x)-1))

WinSize = np.pi/2 #L #/3

Win[abs(xw)<WinSize] = 1.

plt.figure()
plt.plot (xw/np.pi, Win)

s = 25

for i in range (0,n-1):
             
    B[i] = y[i] * Win[i-s+n]                        
       
plt.figure ()    
plt.plot (x/np.pi, B)


## Оконное преобразование 1

In [ ]:
a = np.zeros ((len(x)))
b = np.zeros ((len(x)))
A = np.zeros ((len(x)))

CosTerm = np.zeros ((len(x)))
SinTerm = np.zeros ((len(x)))

k = 2

for s in range (0,n-1): 
    
    a[s]=0.
    b[s]=0.
       
    for i in range (0,n-1):
    
        a[s] +=  2./ WinSize /2 * y[i] * Win[i-s+n] * np.cos(2*np.pi*x[i]/L * k) * dx 
        b[s] +=  2./ WinSize /2 * y[i] * Win[i-s+n] * np.sin(2*np.pi*x[i]/L * k) * dx 
        
        
plt.figure()
plt.plot(x/np.pi, (a**2+b**2)**0.5)
    

## Оконное преобразование 2

In [ ]:
nf = 15

a2 = np.zeros((n,nf))
b2 = np.zeros((n,nf))
A2 = np.zeros((n,nf))

Freq = np.zeros ((len(x)))
Shift = np.zeros ((len(x)))

CosTerm = np.zeros ((len(x)))
SinTerm = np.zeros ((len(x)))

for k in range (0,nf):
    
    Freq [k] = k

    for s in range (0,n-1):
        
        Shift[s] = s
    
        a2[s,k]=0.
        b2[s,k]=0.
       
        for i in range (0,n-1):
    
            a2[s,k] +=  2./ WinSize /2 * y[i] * Win[i-s+n] * np.cos(2*np.pi*x[i]/L * k) * dx 
            b2[s,k] +=  2./ WinSize /2 * y[i] * Win[i-s+n] * np.sin(2*np.pi*x[i]/L * k) * dx
        
                                   
A2 = (a2**2+b2**2)**0.5 


plt.figure()
plt.contourf(x/np.pi, Freq[0:15], A2.T, 100, cmap='rainbow')
plt.colorbar()